In [1]:
%%html
<style type='text/css'>
.CodeMirror{
    font-family: Inconsolata;
    font-size: 13px;
}

div.output_area pre {
    font-family: Inconsolata;
    font-size: 11px;
}

div.text_cell_render {
    font-family: Inconsolata;
    font-size: 13px;
}
</style>

In [16]:
import pandas as pd
import os, shutil
import numpy as np
import matplotlib
import sys; sys.path.append('..')
import pickle
import requests
import torch

from queue import Queue
from tqdm.notebook import tqdm, tqdm_notebook
from collections import defaultdict

path_ddi_dataset = r"/data/data2/041/datasets/DDI"
path_iii_dataset = r"/data/data2/041/datasets/mimic-iii-clinical-database-1.4"

In [17]:
med_pd = pd.read_csv(os.path.join(path_iii_dataset, "PRESCRIPTIONS.csv.gz"), dtype={'NDC':'category'})
med_pd.head()

/data/data2/041/envs/LERS2/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3269: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,00469061711,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,00056017275,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,00338055002,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,00054829725,20mg Tablet,20,mg,1,TAB,PO


In [18]:
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4156450 entries, 0 to 4156449
Data columns (total 19 columns):
 #   Column             Dtype   
---  ------             -----   
 0   ROW_ID             int64   
 1   SUBJECT_ID         int64   
 2   HADM_ID            int64   
 3   ICUSTAY_ID         float64 
 4   STARTDATE          object  
 5   ENDDATE            object  
 6   DRUG_TYPE          object  
 7   DRUG               object  
 8   DRUG_NAME_POE      object  
 9   DRUG_NAME_GENERIC  object  
 10  FORMULARY_DRUG_CD  object  
 11  GSN                object  
 12  NDC                category
 13  PROD_STRENGTH      object  
 14  DOSE_VAL_RX        object  
 15  DOSE_UNIT_RX       object  
 16  FORM_VAL_DISP      object  
 17  FORM_UNIT_DISP     object  
 18  ROUTE              object  
dtypes: category(1), float64(1), int64(3), object(14)
memory usage: 578.9+ MB


# med_process

In [19]:
med_pd.drop(columns=['ROW_ID','DRUG_TYPE','DRUG_NAME_POE','DRUG_NAME_GENERIC',
                    'FORMULARY_DRUG_CD','PROD_STRENGTH','DOSE_VAL_RX',
                    'DOSE_UNIT_RX','FORM_VAL_DISP','FORM_UNIT_DISP', 'GSN', 'FORM_UNIT_DISP',
                    'ROUTE','ENDDATE','DRUG'], axis=1, inplace=True)
med_pd.drop(index = med_pd[med_pd['NDC'] == '0'].index, axis=0, inplace=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3569864 entries, 0 to 4156449
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   SUBJECT_ID  int64   
 1   HADM_ID     int64   
 2   ICUSTAY_ID  float64 
 3   STARTDATE   object  
 4   NDC         category
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 143.1+ MB


In [20]:
med_pd.fillna(method='pad', inplace=True)
med_pd.dropna(inplace=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3569840 entries, 30 to 4156449
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   SUBJECT_ID  int64   
 1   HADM_ID     int64   
 2   ICUSTAY_ID  float64 
 3   STARTDATE   object  
 4   NDC         category
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 143.1+ MB


After dropping many edge features columns, these previous medications recommendation works only consider the NDC identifier of drug, but ignore many variables which describe the dose of drug.

This is why they do following `drop_duplicates`.

In [21]:
med_pd.loc[med_pd.duplicated()]

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,NDC
44,13,143045,263738.0,2167-01-09 00:00:00,00074258702
47,13,143045,263738.0,2167-01-09 00:00:00,00074407532
53,13,143045,263738.0,2167-01-09 00:00:00,00338001702
68,6,107064,263738.0,2175-06-15 00:00:00,00004003822
78,9,150750,263738.0,2149-11-09 00:00:00,00456066270
...,...,...,...,...,...
4156442,98887,121032,238144.0,2144-09-06 00:00:00,61553020648
4156443,98887,121032,238144.0,2144-09-06 00:00:00,61553020648
4156445,98887,121032,238144.0,2144-09-06 00:00:00,00054001820
4156446,98887,121032,238144.0,2144-09-06 00:00:00,00487980125


In [22]:
med_pd.drop_duplicates(inplace=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2987420 entries, 30 to 4156449
Data columns (total 5 columns):
 #   Column      Dtype   
---  ------      -----   
 0   SUBJECT_ID  int64   
 1   HADM_ID     int64   
 2   ICUSTAY_ID  float64 
 3   STARTDATE   object  
 4   NDC         category
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 119.8+ MB


In [23]:
med_pd['ICUSTAY_ID'] = med_pd['ICUSTAY_ID'].astype('int64')
med_pd['STARTDATE'] = pd.to_datetime(med_pd['STARTDATE'], format='%Y-%m-%d %H:%M:%S')
med_pd.sort_values(by=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE'], inplace=True)
med_pd = med_pd.reset_index(drop=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2987420 entries, 0 to 2987419
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   SUBJECT_ID  int64         
 1   HADM_ID     int64         
 2   ICUSTAY_ID  int64         
 3   STARTDATE   datetime64[ns]
 4   NDC         category      
dtypes: category(1), datetime64[ns](1), int64(3)
memory usage: 97.0 MB


In [25]:
med_pd = med_pd.drop(columns=['ICUSTAY_ID'])
med_pd = med_pd.drop_duplicates()
med_pd = med_pd.reset_index(drop=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2961058 entries, 0 to 2961057
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   SUBJECT_ID  int64         
 1   HADM_ID     int64         
 2   STARTDATE   datetime64[ns]
 3   NDC         category      
dtypes: category(1), datetime64[ns](1), int64(2)
memory usage: 73.6 MB


# process_visit_lg2

Filter out patient data with less than two admissions.

In [28]:
a = med_pd[['SUBJECT_ID', 'HADM_ID']].groupby(by='SUBJECT_ID')['HADM_ID'].unique().reset_index()
a

,SUBJECT_ID,HADM_ID
0,2,[163353]
1,4,[185777]
2,6,[107064]
3,8,[159514]
4,9,[150750]
...,...,...
39355,99985,[176670]
39356,99991,[151118]
39357,99992,[197084]
39358,99995,[137810]


In [29]:
a['HADM_ID_Len'] = a['HADM_ID'].map(lambda x:len(x))
a

,SUBJECT_ID,HADM_ID,HADM_ID_Len
0,2,[163353],1
1,4,[185777],1
2,6,[107064],1
3,8,[159514],1
4,9,[150750],1
...,...,...,...
39355,99985,[176670],1
39356,99991,[151118],1
39357,99992,[197084],1
39358,99995,[137810],1


In [31]:
a = a[a['HADM_ID_Len'] > 1]
a

,SUBJECT_ID,HADM_ID,HADM_ID_Len
9,17,"[161087, 194023]",2
13,21,"[109451, 111970]",2
14,23,"[124321, 152223]",2
22,34,"[115799, 144319]",2
24,36,"[122659, 165660, 182104]",3
...,...,...,...
39310,99822,"[146997, 163117, 195871]",3
39328,99883,"[150755, 198523]",2
39331,99897,"[162913, 181057]",2
39338,99923,"[164914, 192053]",2


In [32]:
med_pd_lg2 = a
med_pd_lg2

,SUBJECT_ID,HADM_ID,HADM_ID_Len
9,17,"[161087, 194023]",2
13,21,"[109451, 111970]",2
14,23,"[124321, 152223]",2
22,34,"[115799, 144319]",2
24,36,"[122659, 165660, 182104]",3
...,...,...,...
39310,99822,"[146997, 163117, 195871]",3
39328,99883,"[150755, 198523]",2
39331,99897,"[162913, 181057]",2
39338,99923,"[164914, 192053]",2


In [33]:
med_pd = med_pd.merge(med_pd_lg2[['SUBJECT_ID']], on='SUBJECT_ID', how='inner').reset_index(drop=True)
med_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1135469 entries, 0 to 1135468
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   SUBJECT_ID  1135469 non-null  int64         
 1   HADM_ID     1135469 non-null  int64         
 2   STARTDATE   1135469 non-null  datetime64[ns]
 3   NDC         1135469 non-null  category      
dtypes: category(1), datetime64[ns](1), int64(2)
memory usage: 28.3 MB
